In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import transbigdata as tbd
import warnings
warnings.filterwarnings('ignore')
from utils import centerline
from utils import highway_model
input_networks = '中间数据/network/'
input_abm = '中间数据/abminput/'

INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [2]:
import networkx as nx
import pandas as pd

def compute_directional_distances(edge, node, station2node, stations):
    """
    计算服务区到方向性最近的前后两个充电站和收费站的路网距离。

    参数:
    edges (DataFrame): 包含路网边信息的数据框。
    station2node (DataFrame): 包含站点到节点的映射信息。
    stations (DataFrame): 包含服务区、充电站和收费站的位置信息，需含'flag'列区分类型。

    返回:
    DataFrame: 含有服务区ID和到最近的设施路网距离的数据。
    """
    # 构建有向图
     # find the nearest charge node
    # 构建网络
    G_edges = edge[['u','v','length']].values
    G_nodes = list(node['id'])
    import networkx as nx
    #先创建一个有向图
    G = nx.DiGraph()
    #添加节点
    G.add_nodes_from(G_nodes) 
    #添加边
    G.add_weighted_edges_from(G_edges)

    # 使用station2node映射来获取每个设施的最近节点
    stations = stations.merge(station2node, left_on='id', right_on='station_id', how='right')

    # 筛选出服务区
    service_areas = stations[stations['flag'] == '服务区']
    
    # 计算每个服务区到其他设施的最短路网距离
    distances = []
    for index, service_area in service_areas.iterrows():
        service_node = service_area['node_id']
        
        forward_paths = nx.single_source_dijkstra_path_length(G, service_node, weight='weight')
        reverse_paths = nx.single_source_dijkstra_path_length(G.reverse(), service_node, weight='weight')
        
        facility_distances = stations.copy()
        facility_distances['forward_distance'] = facility_distances['node_id'].map(forward_paths)
        facility_distances['reverse_distance'] = facility_distances['node_id'].map(reverse_paths)

        facility_distances = facility_distances[facility_distances['id'] != service_area['id']]
        
        # 选择前后的充电站和收费站
        facility_distances_forward = facility_distances.sort_values('forward_distance')
        facility_distances_reverse = facility_distances.sort_values('reverse_distance')

        # 对充电站和收费站分别计算
        nearest_forward_charging = facility_distances_forward[facility_distances_forward['flag'] == '服务区'].iloc[0] if not facility_distances_forward.empty else {}
        nearest_reverse_charging = facility_distances_reverse[facility_distances_reverse['flag'] == '服务区'].iloc[0] if not facility_distances_reverse.empty else {}
        nearest_forward_toll = facility_distances_forward[facility_distances_forward['flag'] == '收费站'].iloc[0] if not facility_distances_forward.empty else {}
        nearest_reverse_toll = facility_distances_reverse[facility_distances_reverse['flag'] == '收费站'].iloc[0] if not facility_distances_reverse.empty else {}
        
        distances.append({
            'station_id': service_area['id'],
            # 'nearest_forward_charging_id': nearest_forward_charging.get('id'),
            'nearest_forward_charging_distance': nearest_forward_charging.get('forward_distance'),
            # 'nearest_reverse_charging_id': nearest_reverse_charging.get('id'),
            'nearest_reverse_charging_distance': nearest_reverse_charging.get('reverse_distance'),
            # 'nearest_forward_toll_id': nearest_forward_toll.get('id'),
            'nearest_forward_toll_distance': nearest_forward_toll.get('forward_distance'),
            # 'nearest_reverse_toll_id': nearest_reverse_toll.get('id'),
            'nearest_reverse_toll_distance': nearest_reverse_toll.get('reverse_distance'),
        })

    return pd.DataFrame(distances)

In [3]:
edge = gpd.read_file(input_networks+r'广东省高速公路_简化_边.json')
node = gpd.read_file(input_networks+r'广东省高速公路_简化_节点.json')
station2node = pd.read_csv(input_networks+r'收费站节点对应网络节点的编号.csv')
stations= pd.read_csv(input_networks+r'stations.csv',encoding='gbk')

results = compute_directional_distances(edge, node, station2node, stations)
stations = stations[stations['flag'] == '服务区']

stations = stations.merge(results, left_on='id', right_on='station_id', how='left')
stations = gpd.GeoDataFrame(stations, geometry=gpd.points_from_xy(stations['lon'], stations['lat']))
stations = stations.drop(columns=['flag'])

In [ ]:
import numpy as np
import geopandas as gpd
from mgwr.gwr import GWR, GWRResults
from mgwr.sel_bw import Sel_BW
from libpysal.weights import KNN

# 假设你有一个GeoDataFrame 'gdf'，其中包含了位置坐标和一些解释变量以及因变量

# 准备数据
coords = list(zip(stations ['lon'], stations ['lat']))
y = stations ['orders'].values.reshape((-1,1))  # 因变量
X = stations [['nearest_forward_charging_distance', 'nearest_reverse_charging_distance','nearest_forward_toll_distance','nearest_reverse_toll_distance','充电车位数']].values  # 解释变量

# 选择带宽
bw = Sel_BW(coords, y, X, kernel='bisquare', fixed=False).search()

# 构建模型
model = GWR(coords, y, X, bw, kernel='bisquare', fixed=False)
results = model.fit()

# 查看结果
print(results.summary())
